In [5]:
"""
センサレスベクトル制御 ― 主要モジュール実装例
"""

import numpy as np
from scipy import signal


# ───────────────────────────────────────────────
# 0)  ユーティリティ
# ───────────────────────────────────────────────
def wrap_angle(theta: float) -> float:
    """角度を −π〜π に正規化"""
    return (theta + np.pi) % (2 * np.pi) - np.pi


# ───────────────────────────────────────────────
# 1)  HFV  ― 高周波回転電圧指令ジェネレータ
# ───────────────────────────────────────────────
class HFVGenerator:
    """
    高周波回転電圧 v*1h = Vh [cosφh, sinφh]^T を生成
    """

    def __init__(self, Vh: float, omega_h: float, Ts: float) -> None:
        self.Vh = Vh          # [V]
        self.omega_h = omega_h  # [rad/s]
        self.Ts = Ts
        self.phi_h = 0.0      # 位相レジスタ

    def update(self) -> np.ndarray:
        """1 サンプル分の高周波電圧ベクトルを返す"""
        self.phi_h = wrap_angle(self.phi_h + self.omega_h * self.Ts)
        v_alpha = self.Vh * np.cos(self.phi_h)
        v_beta  = self.Vh * np.sin(self.phi_h)
        return np.array([v_alpha, v_beta], dtype=float)


# ───────────────────────────────────────────────
# 2)  D-module フィルタ  ─ 同相/鏡相電流抽出器
# ───────────────────────────────────────────────
class DModuleFilter:
    def __init__(self, omega_h: float, Ts: float, zeta: float = 0.1) -> None:
        fs = 1 / Ts
        f0 = omega_h / (2 * np.pi)
        bw = f0 * zeta            # 帯域幅の簡易指定
        self.b_i, self.a_i = signal.butter(
            N=2,
            Wn=[f0 - bw, f0 + bw],
            btype="bandpass",
            fs=fs,
        )

        zi0 = signal.lfilter_zi(self.b_i, self.a_i)
        self.zi_alpha = zi0.copy()
        self.zi_beta  = zi0.copy()

    def update(self, i_alpha_beta: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        iα, iβ = i_alpha_beta

        yα, self.zi_alpha = signal.lfilter(
            self.b_i, self.a_i, [iα], zi=self.zi_alpha
        )
        yβ, self.zi_beta = signal.lfilter(
            self.b_i, self.a_i, [iβ], zi=self.zi_beta
        )

        i_hi = np.array([yα[0], yβ[0]])
        i_hm = -i_hi                       # 近似：鏡相 = −同相
        return i_hi, i_hm


# ───────────────────────────────────────────────
# 2')  DModuleFilter2 ― “正統” D因子フィルタ実装
#        ・複素周波数シフト＋ローパスで同相／鏡相を抽出
#        ・論文の F(D)=F(s+jωh), F(s−jωh) に相当
# ───────────────────────────────────────────────
class DModuleFilter2:
    """
    D因子理論に忠実な 2ch フィルタ
    ---------------------------------
    1. αβ 電流を複素数 I = iα + j iβ に変換
    2. 正の回転 e^{−jωh t} を掛けて直流化 → 低域通過  → 同相電流
    3. 負の回転 e^{+jωh t} を掛けて直流化 → 低域通過  → 鏡相電流
       （“鏡”なので周波数符号が逆転）
    4. ベースバンド出力を再び実数ベクトル [α, β] へ戻す
    """

    def __init__(
        self,
        omega_h: float,
        Ts: float,
        lp_order: int = 2,
        lp_cutoff_hz: float | None = None,
    ) -> None:
        """
        Parameters
        ----------
        omega_h : float
            高周波注入角速度 [rad/s]
        Ts : float
            制御周期 [s]
        lp_order : int
            ローパスフィルタ次数（Butterworth）
        lp_cutoff_hz : float | None
            カットオフ周波数 [Hz]  
            未指定なら 0.25·ωh を自動設定（十分に ωh≫fc）
        """
        self.Ts = Ts
        self.omega_h = omega_h
        self.phi = 0.0  # 内部位相レジスタ

        fs = 1.0 / Ts
        if lp_cutoff_hz is None:
            lp_cutoff_hz = (omega_h / (2 * np.pi)) * 0.25  # ωh の 1/4

        # Butterworth デジタル LPF 設計
        self.b_lp, self.a_lp = signal.butter(
            N=lp_order, Wn=lp_cutoff_hz, btype="low", fs=fs
        )

        # lfilter の内部状態（複素数用）
        zi_template = signal.lfilter_zi(self.b_lp, self.a_lp).astype(complex)
        self.zi_i = zi_template.copy()  # in-phase 用
        self.zi_m = zi_template.copy()  # mirror-phase 用

    # ----------------------------------------------------------
    # メイン処理
    # ----------------------------------------------------------
    def update(self, i_alpha_beta: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """
        Parameters
        ----------
        i_alpha_beta : np.ndarray shape(2,)
            ステータ αβ 電流 [A]

        Returns
        -------
        i_hi : np.ndarray shape(2,)  # 同相電流
        i_hm : np.ndarray shape(2,)  # 鏡相電流
        """
        # ① 実 → 複素（α+ jβ）
        i_complex = i_alpha_beta[0] + 1j * i_alpha_beta[1]

        # 内部位相を進める
        self.phi = wrap_angle(self.phi + self.omega_h * self.Ts)
        ejp   = np.exp(-1j * self.phi)  # e^{-jωh t}  … 同相
        ejm   = np.exp(+1j * self.phi)  # e^{+jωh t}  … 鏡相

        # ② 周波数シフト
        x_i = i_complex * ejp   # 正シフト で +ωh を 0 へ
        x_m = i_complex * ejm   # 負シフト で −ωh を 0 へ

        # ③ LPF 通過
        y_i, self.zi_i = signal.lfilter(self.b_lp, self.a_lp, [x_i], zi=self.zi_i)
        y_m, self.zi_m = signal.lfilter(self.b_lp, self.a_lp, [x_m], zi=self.zi_m)

        # ④ 複素 → 実ベクトルに戻す（α=Re, β=Im）
        i_hi = np.array([y_i.real[0], y_i.imag[0]])
        i_hm = np.array([y_m.real[0], y_m.imag[0]])

        return i_hi, i_hm


# ───────────────────────────────────────────────
# 3)  Phase Estimator（D-module + PLL）
# ───────────────────────────────────────────────
class PhaseEstimator:
    """
    ・DModuleFilter で同相/鏡相電流を抽出
    ・PLL（比例＋積分）で電気角と電気角速度を推定
    """

    def __init__(
        self,
        omega_h: float,
        Ts: float,
        Kp: float = 20.0,
        Ki: float = 400.0,
        zeta: float = 0.1,
    ) -> None:
        self.Ts = Ts
        # self.filter = DModuleFilter(omega_h, Ts, zeta)
        self.filter = DModuleFilter2(omega_h, Ts, lp_order=2)
        self.Kp = Kp
        self.Ki = Ki
        self.theta_e = 0.0  # 電気角推定値
        self.omega_e = 0.0  # 電気角速度推定値
        self.err_int = 0.0  # 誤差積分

    def _phase_error(self, i_hi: np.ndarray) -> float:
        """
        PLL 位相誤差計算
        dq 座標で d 成分（推定座標系）を負符号で返す
        """
        # αβ → dq 変換（推定角で Park 変換）
        ca, sa = np.cos(self.theta_e), np.sin(self.theta_e)
        i_d =  ca * i_hi[0] + sa * i_hi[1]
        # PLL 用誤差は −i_d とする（文献式 18 相当）
        return -i_d

    def update(self, i_alpha_beta: np.ndarray) -> tuple[float, float, float]:
        """
        Returns
        -------
        theta_e  : 電気角 [rad]
        sinθ, cosθ : sin/cos(θ̂) （ベクトル回転器用）
        """
        i_hi, i_hm = self.filter.update(i_alpha_beta)
        err = self._phase_error(i_hi)

        # PI 制御
        self.err_int += err * self.Ts
        self.omega_e = self.Kp * err + self.Ki * self.err_int
        self.theta_e = wrap_angle(self.theta_e + self.omega_e * self.Ts)

        return self.theta_e, np.sin(self.theta_e), np.cos(self.theta_e), self.omega_e


# ───────────────────────────────────────────────
# 4)  Speed Estimator（極対数で割るだけ）
# ───────────────────────────────────────────────
class SpeedEstimator:
    def __init__(self, pole_pairs: int) -> None:
        self.Np = pole_pairs

    def update(self, omega_e: float) -> float:
        """機械角速度 [rad/s] を返す"""
        return omega_e / self.Np


# ───────────────────────────────────────────────
# 5)  Fbs ― バンドストップ（ノッチ）フィルタ
# ───────────────────────────────────────────────
class BandStopFilter:
    """
    IIR ノッチフィルタで高周波成分を除去
    Q 値（鋭さ）が高いほど帯域が狭い
    """

    def __init__(self, omega_h: float, Ts: float, Q: float = 15.0) -> None:
        fs = 1 / Ts
        f0 = omega_h / (2 * np.pi)  # [Hz]
        b, a = signal.iirnotch(w0=f0, Q=Q, fs=fs)
        self.b, self.a = b, a
        self.zi = [signal.lfilter_zi(b, a).copy() for _ in range(2)]  # α,β 用

    def update(self, dq: np.ndarray) -> np.ndarray:
        """dq = [d, q]"""
        d, self.zi[0] = signal.lfilter(self.b, self.a, [dq[0]], zi=self.zi[0])
        q, self.zi[1] = signal.lfilter(self.b, self.a, [dq[1]], zi=self.zi[1])
        return np.array([d[0], q[0]])


# ───────────────────────────────────────────────
# 6)  サンプルメインループ（テスト用途）
# ───────────────────────────────────────────────
if __name__ == "__main__":
    # ==== パラメータ ====
    Ts = 1.0e-4          # 10 kHz 制御周期
    omega_h = 2 * np.pi * 400.0  # 高周波 400 Hz
    Vh = 20.0            # 高周波電圧振幅
    pole_pairs = 4

    # ==== モジュール生成 ====
    hfv = HFVGenerator(Vh, omega_h, Ts)
    phase_est = PhaseEstimator(omega_h, Ts)
    speed_est = SpeedEstimator(pole_pairs)
    notch = BandStopFilter(omega_h, Ts)
    
    
    
    
    
    

    # デバッグ用のダミー電流（ここではゼロにノイズを重畳）
    rng = np.random.default_rng(seed=0)

    # ==== 1 秒シミュレーション ====
    steps = int(1.0 / Ts)
    for k in range(steps):
        # ダミー：iαβ に微小ノイズを与える
        i_αβ = 0.01 * rng.standard_normal(2)

        # 1) HFV 生成
        v_hf = hfv.update()

        # （ここで本来は PWM → モータモデル → iαβ を取得）

        # 2) 位相推定
        theta, s, c, omega_e = phase_est.update(i_αβ)

        # 3) 速度推定
        omega_m = speed_est.update(omega_e)

        # 4) Notch フィルタ（dq 電流に適用する例）
        #   ここでは d=q=0 としてデモ
        iq_dq = np.array([0.0, 0.0])
        iq_dq_filtered = notch.update(iq_dq)

        # ---- ログや可視化は省略 ----

    print("シミュレーション完了")


シミュレーション完了
